# 1.数据清洗

In [1]:
#导入包
#如何下载奇怪的库
#win+R+cmd+https://blog.csdn.net/aobulaien001/article/details/133298563
##python第三方库下载：https://pypi.org/
import pandas as pd
import numpy as np
# from gensim.corpora import Dictionary
# from gensim import corpora, models
import re#中文去标点符号库##用法：https://blog.csdn.net/spring_willow/article/details/80211243
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)#LDA文本模型
from tqdm import tqdm#为使用process_apply
import jieba
tqdm.pandas()
import codecs#停用词相关
from sklearn.feature_extraction.text import CountVectorizer#TF-IDF
### TF-IDF到LDA建模相关##https://zhuanlan.zhihu.com/p/372269493
import pyLDAvis#pip install pyLDAvis -i https://pypi.tuna.tsinghua.edu.cn/simple/
#pip install pyLDAvis scikit-learn -i https://pypi.tuna.tsinghua.edu.cn/simple/
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer#lda输出结果变成df相关
from sklearn.decomposition import LatentDirichletAllocation#lda输出结果变成df相关
import os
###

C:\Users\celia\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\celia\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
#pip install pyLDAvis

In [3]:
import pyLDAvis.lda_model

In [4]:
comment_20 = pd.read_csv('数据\\2020_2021微信公众号新闻.csv',encoding='gbk').astype(str)#导入20年数据
comment_22 = pd.read_csv('数据\\2022_2023微信公众号新闻.csv',encoding='utf-8').astype(str)#导入22年评论数据
comment20_text = pd.DataFrame()
comment22_text = pd.DataFrame()

In [5]:
comment_20['txt'] = comment_20['正文'] + ' ' + comment_20['公众号标题']#合并标题正文
comment_22['txt'] = comment_22['正文'] + ' ' + comment_22['公众号标题']#合并标题正文
comment_20


,文章ID,公众号标题,发布时间,正文,txt
0,1001,2018，对自己好一点,2018/1/2 17:28,2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，请对自己好一点\n一辈子很...,2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，请对自己好一点\n一辈子很...
1,1002,春节机票预订有窍门,2018/1/2 17:28,距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚至一票难求。在这里分享...,距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚至一票难求。在这里分享...
2,1003,冬日旅游知多D,2018/1/3 17:32,960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\n但是冬季的严寒气...,960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\n但是冬季的严寒气...
3,1004,2018冬季暖心之旅,2018/1/3 17:32,长按二维码，关注我们\n中心联系人：林小姐13709649096\n刘小姐135000781...,长按二维码，关注我们\n中心联系人：林小姐13709649096\n刘小姐135000781...
4,1005,关于粤K27618号大客车排气管“喷火”事件的情况说明,2018/1/5 16:57,,关于...
...,...,...,...,...,...
616,1617,北部湾城市运动会我市代表队喜获佳绩,2019/12/25 11:38,由北部湾经济合作组织、玉林市人民政府主办的第二届北部湾城市运动会于2019年12月6日至16...,由北部湾经济合作组织、玉林市人民政府主办的第二届北部湾城市运动会于2019年12月6日至16...
617,1618,茂名市交通建设投资集团有限公司新年文艺晚会现场直播，敬请关注！,2019/12/25 18:00,收看现场直播请点击文章最后面：阅读原文\n开播时间：2019年12月26日 20:00\n...,收看现场直播请点击文章最后面：阅读原文\n开播时间：2019年12月26日 20:00\n...
618,1619,北部湾城市运动会，茂名代表队喜获佳绩！,2019/12/25 21:54,由北部湾经济合作组织、玉林市人民政府主办的第二届北部湾城市运动会于2019年12月6日至16...,由北部湾经济合作组织、玉林市人民政府主办的第二届北部湾城市运动会于2019年12月6日至16...
619,1620,睇年例、游滨海……这个冬天，就到茂名过大年！,2019/12/27 21:00,茂名，一座面朝大海、四季花开的好心之城。一座全年空气质量优良天数超过97%的氧吧城市。茂名的...,茂名，一座面朝大海、四季花开的好心之城。一座全年空气质量优良天数超过97%的氧吧城市。茂名的...


In [6]:
#定义文本清洗函数
def cleanmytxt(t):
    if t != '':
        t = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、“”~@#￥%……&*（）【】=+：；！@#￥%%……&*（）{}~·《》/ ]1234567890", "",t)#中文
        t =  re.sub("~!@#$%^&*()_+{}|<>?`-=\;',./'", "",t)#清理英文字符
        tlist = jieba.cut(t,cut_all=False)#分词
        tsentence = ''
        #分词后连接
        for word in tlist:
            if word != '\t':
                tsentence += word + " "
    return tsentence.strip()

<>:4: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \;
<>:4: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \;
C:\Users\celia\AppData\Local\Temp/ipykernel_11024/566379190.py:4: DeprecationWarning: invalid escape sequence \s
  t = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、“”~@#￥%……&*（）【】=+：；！@#￥%%……&*（）{}~·《》/ ]1234567890", "",t)#中文
C:\Users\celia\AppData\Local\Temp/ipykernel_11024/566379190.py:5: DeprecationWarning: invalid escape sequence \;
  t =  re.sub("~!@#$%^&*()_+{}|<>?`-=\;',./'", "",t)#清理英文字符


In [7]:
data1 = comment_20['txt'].progress_apply(cleanmytxt)#process_apply为进度条

  0%|                                                                                          | 0/621 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\celia\AppData\Local\Temp\jieba.cache
Loading model cost 1.275 seconds.
Prefix dict has been built successfully.
100%|████████████████████████████████████████████████████████████████████████████████| 621/621 [00:07<00:00, 80.27it/s]


In [8]:
data2 = comment_22['txt'].progress_apply(cleanmytxt)

100%|█████████████████████████████████████████████████████████████████████████████| 5665/5665 [00:53<00:00, 105.69it/s]


In [9]:
data1

0      2017 的 旅程 已经 结束 2018 的 未来 拉开 了 帷幕 新 的 一年 里 ， 请...
1      距离 春节 还有 一个多月 的 时间 ， 在线 旅游 网站 的 春节 机票 销售 火爆 ， ...
2      960 万平方公里 的 祖国 大地 ， 四季 都 有 独特 美景 冬天 的 旅行 也 别 有...
3      长 按 二维码 ， 关注 我们 中心 联系人 ： 林小姐 13709649096 刘小姐 1...
4                关于 粤 K27618 号 大客车 排气管 “ 喷火 ” 事件 的 情况 说明
                             ...                        
616    由 北部湾 经济合作组织 、 玉林市 人民政府 主办 的 第二届 北部湾 城市 运动会 于 ...
617    收看 现场直播 请 点击 文章 最后 面 ： 阅读 原文 开播 时间 ： 2019 年 12...
618    由 北部湾 经济合作组织 、 玉林市 人民政府 主办 的 第二届 北部湾 城市 运动会 于 ...
619    茂名 ， 一座 面朝 大海 、 四季 花开 的 好心 之 城 。 一座 全年 空气质量 优良...
620    茂名 交投 集团 — — 鲲鹏展翅 德行 天下 2019 年 12 月 28 日 （ 今天 ...
Name: txt, Length: 621, dtype: object

## 去停用词

In [10]:
import codecs#停用词相关
stopword_list = [k.strip() for k in open('数据\\chineseStopWords.txt', encoding='utf8').readlines() if k.strip() != '']
#读取停用词列表
def stopWord(line):
    sentence = delstopword(line, stopword_list)
    return sentence
# #删除停用词
def delstopword(line, stopkey):
    wordList = line.split(' ')
    sentence = ''
    for word in wordList:
        word = word.strip()
        if word not in stopkey:
            if word != '\t':
                sentence += word + " "
    return sentence.strip()

In [11]:
comment_20['qtyc'] = data1
comment_22['qtyc'] = data2
txt1 = comment_20['qtyc'].progress_apply(stopWord)
txt2 = comment_22['qtyc'].progress_apply(stopWord)


100%|██████████████████████████████████████████████████████████████████████████████| 5665/5665 [01:07<00:00, 84.17it/s]


# 2.各产业分析

## 2.1 旅游产品标签化

### 酒店

In [12]:
hotel_pl_20 = pd.read_csv('数据\\2020_2021酒店评论.csv',encoding='gbk').astype(str)#导入20年数据
hotel_pl_22 = pd.read_csv('数据\\2022_2023酒店评论.csv',encoding='gbk').astype(str)#导入22年数据
hotel_pl= pd.concat([hotel_pl_20,hotel_pl_22])
hotel_pl

,酒店评论ID,城市,酒店名称,评论日期,评论内容,入住日期,入住房型
0,1001,茂名,茂名君悦商务酒店,2018/12/2,干净卫生服务好,2018/12/2,标双
1,1002,茂名,维也纳国际酒店(茂名电白店),2018/12/3,环境可以，干净！,2018/12/3,豪华双床房
2,1003,茂名,茂名永利之家,2018/12/4,环境不错，房间卫生都很好，生活也很方便，就是隔音效果不理想，有时太吵。我定的优惠价，性价比很...,2018/12/4,标准单人间
3,1004,茂名,茂名诚荟酒店,2018/12/5,很好.......舒服态度不错,2018/12/5,豪华大床房
4,1005,茂名,茂名华景商务酒店,2018/12/6,#卫生# #设计风格# #酒店餐饮#,2018/12/6,特惠单人间(无窗)
...,...,...,...,...,...,...,...
694,2089,茂名,凯里亚德酒店(茂名高铁站店),2021/11/27,酒店位置和环境都不错，而且非常新！,2021/11/27,优享双床房
695,2090,茂名,麗枫酒店(茂名电白万达广场店),2021/11/28,kf 服务：客房服务态度好 很认真,2021/11/28,豪华大床房
696,2091,茂名,茂名观海楼假日酒店,2021/11/29,酒店有专属停车场，停车方便，房间挺大，风景挺好，下楼就是沙滩,2021/11/29,无敌海景双床房
697,2092,茂名,茂名御景国际大酒店,2021/11/30,酒店在电白来说比较高档环境还可以 出行方便 交通：方便停车免费停车挺好的 免费健身房可以游...,2021/11/30,行政套房


In [13]:
hotel_pl.info()#查看属性

<class 'pandas.core.frame.DataFrame'>
Index: 1093 entries, 0 to 698
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   酒店评论ID  1093 non-null   object
 1   城市      1093 non-null   object
 2   酒店名称    1093 non-null   object
 3   评论日期    1093 non-null   object
 4   评论内容    1093 non-null   object
 5   入住日期    1093 non-null   object
 6   入住房型    1093 non-null   object
dtypes: object(7)
memory usage: 68.3+ KB


In [14]:
#大致去重
reviews=hotel_pl.copy()
print('去重之前,',reviews.shape[0])
reviews_hotel=reviews.drop_duplicates([ '评论内容'])
print('去重之后,',reviews.shape[0])

去重之前, 1093
去重之后, 1093


In [15]:
reviews_hotel_id = reviews_hotel['酒店评论ID']
reviews_hotel_name = reviews_hotel['酒店名称']
reviews_hotel_pl = reviews_hotel['评论内容']
reviews_hotel_date =reviews_hotel['评论日期']
reviews_hotel = pd.concat((reviews_hotel_id,reviews_hotel_name,reviews_hotel_date,reviews_hotel_pl),axis=1,join ='inner')
reviews_hotel

,酒店评论ID,酒店名称,评论日期,评论内容
0,1001,茂名君悦商务酒店,2018/12/2,干净卫生服务好
1,1002,维也纳国际酒店(茂名电白店),2018/12/3,环境可以，干净！
2,1003,茂名永利之家,2018/12/4,环境不错，房间卫生都很好，生活也很方便，就是隔音效果不理想，有时太吵。我定的优惠价，性价比很...
3,1004,茂名诚荟酒店,2018/12/5,很好.......舒服态度不错
4,1005,茂名华景商务酒店,2018/12/6,#卫生# #设计风格# #酒店餐饮#
...,...,...,...,...
694,2089,凯里亚德酒店(茂名高铁站店),2021/11/27,酒店位置和环境都不错，而且非常新！
695,2090,麗枫酒店(茂名电白万达广场店),2021/11/28,kf 服务：客房服务态度好 很认真
696,2091,茂名观海楼假日酒店,2021/11/29,酒店有专属停车场，停车方便，房间挺大，风景挺好，下楼就是沙滩
697,2092,茂名御景国际大酒店,2021/11/30,酒店在电白来说比较高档环境还可以 出行方便 交通：方便停车免费停车挺好的 免费健身房可以游...


In [16]:
reviews_hotel['酒店评论ID'] = reviews_hotel['酒店评论ID']+'-'+reviews_hotel['酒店名称'].map(str)
reviews_hotel=reviews_hotel.sort_values(['酒店名称'])#按照名称安排在一起
reviews_hotel_name_qc =reviews_hotel.drop_duplicates(['酒店名称'])
reviews_hotel_name_qc

,酒店评论ID,酒店名称,评论日期,评论内容
31,1032-52连锁酒店(化州店),52连锁酒店(化州店),2019/1/2,酒店装饰很温馨，店员也很热情，位置正市中心，方便快捷，遇到一些小事故，店员配合很好，性价比很...
212,1213-IU酒店(信宜中兴六路店),IU酒店(信宜中兴六路店),2019/7/3,环境还是很不错的，卫生也很干净，
324,1325-IU酒店(茂名人民南路油城大厦店),IU酒店(茂名人民南路油城大厦店),2019/10/23,环境各方面都还好 就是办事效率有点低 而且给人不专业的感觉 我们办理入住的时候已经是凌晨了 ...
338,1733-Q加·茂名大喜假日酒店,Q加·茂名大喜假日酒店,2020/12/6,位置还是不错的，在新湖二路，整条街都是酒店。上次5月份就是住的这里，住了好几天，感觉还不错，...
172,1567-信宜丽晶酒店,信宜丽晶酒店,2020/6/23,还不错空调比维也纳的要好
...,...,...,...,...
126,1521-高州简尚精品酒店,高州简尚精品酒店,2020/5/8,酒店新净，卫生也清洁
34,1429-高州顺得商务酒店,高州顺得商务酒店,2020/2/5,门口可以停车，车位不多，就在一条热闹的街边，吃饭逛街购物很方便。
687,2082-麗枫酒店(茂名水东店),麗枫酒店(茂名水东店),2021/11/20,交通：很方便 卫生：很舒服住的 环境：很美 服务：特别好，我都给前台小姐姐好评，他们特别很好...
218,1613-麗枫酒店(茂名电白万达广场店),麗枫酒店(茂名电白万达广场店),2020/8/8,第一次体验丽枫酒店，到门口的时候震惊到了，地理位置超好，（旁边就是万达广场）停车场很宽敞，门...


In [17]:
print('去重之前酒店店铺数量,',reviews_hotel.shape[0])
print('去重之后酒店店铺数量,',reviews_hotel_name_qc.shape[0])

去重之前酒店店铺数量, 1086
去重之后酒店店铺数量, 89


In [18]:
hotel_name = reviews_hotel_name_qc['酒店名称']
hotel_name

31           52连锁酒店(化州店)
212        IU酒店(信宜中兴六路店)
324    IU酒店(茂名人民南路油城大厦店)
338          Q加·茂名大喜假日酒店
172               信宜丽晶酒店
             ...        
126             高州简尚精品酒店
34              高州顺得商务酒店
687          麗枫酒店(茂名水东店)
218      麗枫酒店(茂名电白万达广场店)
47          麗枫酒店(茂名高铁站店)
Name: 酒店名称, Length: 89, dtype: object

In [19]:
hotel_number = list(range(1,90))
connect_hotel = dict(zip(hotel_name,hotel_number))
reviews_hotel['产品ID']= 0
reviews_hotel
for i in range(1086):
    I = reviews_hotel.iloc[i,1]
    reviews_hotel.iloc[i,4] = connect_hotel[I]

In [20]:
reviews_hotel['ID'] = 'ID'
reviews_hotel['产品ID'] = reviews_hotel['ID'].map(str) + reviews_hotel['产品ID'].map(str)
reviews_hotel

,酒店评论ID,酒店名称,评论日期,评论内容,产品ID,ID
31,1032-52连锁酒店(化州店),52连锁酒店(化州店),2019/1/2,酒店装饰很温馨，店员也很热情，位置正市中心，方便快捷，遇到一些小事故，店员配合很好，性价比很...,ID1,ID
45,1046-52连锁酒店(化州店),52连锁酒店(化州店),2019/1/16,各方面都不错。02服务员的态度真心不错。,ID1,ID
49,1050-52连锁酒店(化州店),52连锁酒店(化州店),2019/1/20,已经是老顾客了，每一次都不让我失望，服务很热情，最喜欢房间的布置了，价格也实惠，祝老板生意兴隆。,ID1,ID
212,1213-IU酒店(信宜中兴六路店),IU酒店(信宜中兴六路店),2019/7/3,环境还是很不错的，卫生也很干净，,ID2,ID
324,1325-IU酒店(茂名人民南路油城大厦店),IU酒店(茂名人民南路油城大厦店),2019/10/23,环境各方面都还好 就是办事效率有点低 而且给人不专业的感觉 我们办理入住的时候已经是凌晨了 ...,ID3,ID
...,...,...,...,...,...,...
621,2016-麗枫酒店(茂名电白万达广场店),麗枫酒店(茂名电白万达广场店),2021/9/15,服务态度好 不错不错,ID88,ID
632,2027-麗枫酒店(茂名电白万达广场店),麗枫酒店(茂名电白万达广场店),2021/9/26,连住两晚，到晏镜岭、架海灯塔分别40分钟，1小时20分钟。停车方便，设施尚可。前台服务不错。...,ID88,ID
513,1908-麗枫酒店(茂名电白万达广场店),麗枫酒店(茂名电白万达广场店),2021/5/30,有停车场，很方便，酒店很干净！挺新的。而且这家店还很有爱！,ID88,ID
47,1442-麗枫酒店(茂名高铁站店),麗枫酒店(茂名高铁站店),2020/2/19,服务态度，环境各方面挺好,ID89,ID


#### 评论内容清洗

In [21]:
hotel = reviews_hotel['评论内容'].progress_apply(cleanmytxt)#process_apply为进度条

100%|████████████████████████████████████████████████████████████████████████████| 1086/1086 [00:00<00:00, 3105.58it/s]


In [22]:
hotel

31     酒店 装饰 很 温馨 ， 店员 也 很 热情 ， 位置 正 市中心 ， 方便快捷 ， 遇到 ...
45                       各 方面 都 不错 。 02 服务员 的 态度 真心 不错 。
49     已经 是 老 顾客 了 ， 每 一次 都 不让 我 失望 ， 服务 很 热情 ， 最 喜欢 ...
212                           环境 还是 很 不错 的 ， 卫生 也 很 干净 ，
324    环境 各 方面 都 还好 就是 办事效率 有点 低 而且 给 人 不 专业 的 感觉 我们 ...
                             ...                        
621                                         服务态度 好 不错 不错
632    连住 两晚 ， 到 晏镜岭 、 架海 灯塔 分别 40 分钟 ， 1 小时 20 分钟 。 ...
513    有 停车场 ， 很 方便 ， 酒店 很 干净 ！ 挺 新 的 。 而且 这家 店 还 很 有...
47                                    服务态度 ， 环境 各 方面 挺 好
45                                      服务 很 好 ， 卫生 很 干净
Name: 评论内容, Length: 1086, dtype: object

In [23]:
hotel = hotel.progress_apply(stopWord)#去停用词

100%|████████████████████████████████████████████████████████████████████████████| 1086/1086 [00:00<00:00, 2225.75it/s]


In [24]:
reviews_hotel['分词并去停用词'] = hotel

In [25]:
reviews_hotel

,酒店评论ID,酒店名称,评论日期,评论内容,产品ID,ID,分词并去停用词
31,1032-52连锁酒店(化州店),52连锁酒店(化州店),2019/1/2,酒店装饰很温馨，店员也很热情，位置正市中心，方便快捷，遇到一些小事故，店员配合很好，性价比很...,ID1,ID,酒店 装饰 温馨 店员 热情 位置 正 市中心 方便快捷 事故 店员 配合 好 性价比 高 酒店
45,1046-52连锁酒店(化州店),52连锁酒店(化州店),2019/1/16,各方面都不错。02服务员的态度真心不错。,ID1,ID,不错 02 服务员 态度 真心 不错
49,1050-52连锁酒店(化州店),52连锁酒店(化州店),2019/1/20,已经是老顾客了，每一次都不让我失望，服务很热情，最喜欢房间的布置了，价格也实惠，祝老板生意兴隆。,ID1,ID,老 顾客 不让 失望 服务 热情 喜欢 房间 布置 价格 实惠 祝 老板 生意兴隆
212,1213-IU酒店(信宜中兴六路店),IU酒店(信宜中兴六路店),2019/7/3,环境还是很不错的，卫生也很干净，,ID2,ID,环境 不错 卫生 干净
324,1325-IU酒店(茂名人民南路油城大厦店),IU酒店(茂名人民南路油城大厦店),2019/10/23,环境各方面都还好 就是办事效率有点低 而且给人不专业的感觉 我们办理入住的时候已经是凌晨了 ...,ID3,ID,环境 还好 办事效率 低 专业 感觉 办理 入住 凌晨 累 前台 登记 将近 二十多分钟 没...
...,...,...,...,...,...,...,...
621,2016-麗枫酒店(茂名电白万达广场店),麗枫酒店(茂名电白万达广场店),2021/9/15,服务态度好 不错不错,ID88,ID,服务态度 好 不错 不错
632,2027-麗枫酒店(茂名电白万达广场店),麗枫酒店(茂名电白万达广场店),2021/9/26,连住两晚，到晏镜岭、架海灯塔分别40分钟，1小时20分钟。停车方便，设施尚可。前台服务不错。...,ID88,ID,连住 两晚 晏镜岭 架海 灯塔 40 分钟 1 小时 20 分钟 停车 设施 尚可 前台 服...
513,1908-麗枫酒店(茂名电白万达广场店),麗枫酒店(茂名电白万达广场店),2021/5/30,有停车场，很方便，酒店很干净！挺新的。而且这家店还很有爱！,ID88,ID,停车场 酒店 干净 挺 新 这家 店 爱
47,1442-麗枫酒店(茂名高铁站店),麗枫酒店(茂名高铁站店),2020/2/19,服务态度，环境各方面挺好,ID89,ID,服务态度 环境 挺 好


In [76]:
##情感分析
hh = reviews_hotel.drop('ID',axis=1)
hh = hh.drop('评论日期',axis=1)
hh = hh.drop('酒店名称',axis=1)
hh = hh.drop('分词并去停用词',axis=1)
hh

,酒店评论ID,评论内容,产品ID
31,1032-52连锁酒店(化州店),酒店装饰很温馨，店员也很热情，位置正市中心，方便快捷，遇到一些小事故，店员配合很好，性价比很...,ID1
45,1046-52连锁酒店(化州店),各方面都不错。02服务员的态度真心不错。,ID1
49,1050-52连锁酒店(化州店),已经是老顾客了，每一次都不让我失望，服务很热情，最喜欢房间的布置了，价格也实惠，祝老板生意兴隆。,ID1
212,1213-IU酒店(信宜中兴六路店),环境还是很不错的，卫生也很干净，,ID2
324,1325-IU酒店(茂名人民南路油城大厦店),环境各方面都还好 就是办事效率有点低 而且给人不专业的感觉 我们办理入住的时候已经是凌晨了 ...,ID3
...,...,...,...
621,2016-麗枫酒店(茂名电白万达广场店),服务态度好 不错不错,ID88
632,2027-麗枫酒店(茂名电白万达广场店),连住两晚，到晏镜岭、架海灯塔分别40分钟，1小时20分钟。停车方便，设施尚可。前台服务不错。...,ID88
513,1908-麗枫酒店(茂名电白万达广场店),有停车场，很方便，酒店很干净！挺新的。而且这家店还很有爱！,ID88
47,1442-麗枫酒店(茂名高铁站店),服务态度，环境各方面挺好,ID89


In [78]:
hh[['酒店评论ID','产品ID']]=hh[['产品ID','酒店评论ID']]
hh.columns = ['语料ID','产品ID','产品名称']
hh

,语料ID,产品ID,产品名称
31,ID1,酒店装饰很温馨，店员也很热情，位置正市中心，方便快捷，遇到一些小事故，店员配合很好，性价比很...,1032-52连锁酒店(化州店)
45,ID1,各方面都不错。02服务员的态度真心不错。,1046-52连锁酒店(化州店)
49,ID1,已经是老顾客了，每一次都不让我失望，服务很热情，最喜欢房间的布置了，价格也实惠，祝老板生意兴隆。,1050-52连锁酒店(化州店)
212,ID2,环境还是很不错的，卫生也很干净，,1213-IU酒店(信宜中兴六路店)
324,ID3,环境各方面都还好 就是办事效率有点低 而且给人不专业的感觉 我们办理入住的时候已经是凌晨了 ...,1325-IU酒店(茂名人民南路油城大厦店)
...,...,...,...
621,ID88,服务态度好 不错不错,2016-麗枫酒店(茂名电白万达广场店)
632,ID88,连住两晚，到晏镜岭、架海灯塔分别40分钟，1小时20分钟。停车方便，设施尚可。前台服务不错。...,2027-麗枫酒店(茂名电白万达广场店)
513,ID88,有停车场，很方便，酒店很干净！挺新的。而且这家店还很有爱！,1908-麗枫酒店(茂名电白万达广场店)
47,ID89,服务态度，环境各方面挺好,1442-麗枫酒店(茂名高铁站店)


In [80]:
comment_hotel = reviews_hotel['评论内容'].progress_apply(cleanmytxt)
comment_hotel

100%|████████████████████████████████████████████████████████████████████████████| 1086/1086 [00:01<00:00, 1041.01it/s]


31     酒店 装饰 很 温馨 ， 店员 也 很 热情 ， 位置 正 市中心 ， 方便快捷 ， 遇到 ...
45                       各 方面 都 不错 。 02 服务员 的 态度 真心 不错 。
49     已经 是 老 顾客 了 ， 每 一次 都 不让 我 失望 ， 服务 很 热情 ， 最 喜欢 ...
212                           环境 还是 很 不错 的 ， 卫生 也 很 干净 ，
324    环境 各 方面 都 还好 就是 办事效率 有点 低 而且 给 人 不 专业 的 感觉 我们 ...
                             ...                        
621                                         服务态度 好 不错 不错
632    连住 两晚 ， 到 晏镜岭 、 架海 灯塔 分别 40 分钟 ， 1 小时 20 分钟 。 ...
513    有 停车场 ， 很 方便 ， 酒店 很 干净 ！ 挺 新 的 。 而且 这家 店 还 很 有...
47                                    服务态度 ， 环境 各 方面 挺 好
45                                      服务 很 好 ， 卫生 很 干净
Name: 评论内容, Length: 1086, dtype: object

In [90]:
comment_hotel.to_csv('酒店分词结果.csv',encoding='utf_8_sig',index=False)
neg_pos_hotel=pd.read_csv('数据\\qyh全部数据\\全部数据\\red_pol_1.csv',encoding="gbk").astype(str)
neg_pos_hotel = neg_pos_hotel.drop('Unnamed: 0',axis=1)
neg_pos_hotel

,neg_score,pos_score,产品ID
0,1,4,ID1
1,0,2,ID1
2,1,6,ID1
3,0,2,ID2
4,0,0,ID3
...,...,...,...
1088,0,11,ID88
1089,2,12,ID88
1090,0,1,ID88
1091,0,2,ID89


In [91]:
reviews_hotel = reviews_hotel.reset_index(drop=True)
neg_pos_hotel['评论日期'] = reviews_hotel['评论日期']
neg_pos_hotel

,neg_score,pos_score,产品ID,评论日期
0,1,4,ID1,2019/1/2
1,0,2,ID1,2019/1/16
2,1,6,ID1,2019/1/20
3,0,2,ID2,2019/7/3
4,0,0,ID3,2019/10/23
...,...,...,...,...
1088,0,11,ID88,NaN
1089,2,12,ID88,NaN
1090,0,1,ID88,NaN
1091,0,2,ID89,NaN


In [92]:
neg_pos_hotel['年份'] = 0
for i in range(0,1093):
    neg_pos_hotel.iloc[i,4] = str(neg_pos_hotel.iloc[i,3])[0:4]
neg_pos_hotel

C:\Users\celia\AppData\Local\Temp/ipykernel_11024/3985177321.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2019' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  neg_pos_hotel.iloc[i,4] = str(neg_pos_hotel.iloc[i,3])[0:4]


,neg_score,pos_score,产品ID,评论日期,年份
0,1,4,ID1,2019/1/2,2019
1,0,2,ID1,2019/1/16,2019
2,1,6,ID1,2019/1/20,2019
3,0,2,ID2,2019/7/3,2019
4,0,0,ID3,2019/10/23,2019
...,...,...,...,...,...
1088,0,11,ID88,NaN,nan
1089,2,12,ID88,NaN,nan
1090,0,1,ID88,NaN,nan
1091,0,2,ID89,NaN,nan


In [94]:
q = pd.DataFrame({'neg_score':range(0,200),'pos_score':range(0,200),'产品ID':range(0,200),'年份':range(0,200),'间隔':range(0,200),'次数':range(0,200)})
q['neg_score'] = 0
q['pos_score'] = 0
q['年份'] = 0
q['产品ID'] = 0
q['间隔'] = 0
q['次数'] = 0
q

,neg_score,pos_score,产品ID,年份,间隔,次数
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
195,0,0,0,0,0,0
196,0,0,0,0,0,0
197,0,0,0,0,0,0
198,0,0,0,0,0,0


In [97]:
q.iloc[0,0]=neg_pos_hotel.iloc[0,0]
q.iloc[0,1]=neg_pos_hotel.iloc[0,1]
q.iloc[0,2]=neg_pos_hotel.iloc[0,2]
q.iloc[0,3]=neg_pos_hotel.iloc[0,4]
q

,neg_score,pos_score,产品ID,年份,间隔,次数
0,1,4,ID1,2019,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
195,0,0,0,0,0,0
196,0,0,0,0,0,0
197,0,0,0,0,0,0
198,0,0,0,0,0,0


In [98]:
from datetime import datetime
def daysBetweenDates(date1, date2):
    y1, m1, d1 = date1.split("-")
    y2, m2, d2 = date2.split("-")
    cur_day = datetime(int(y1), int(m1), int(d1))
    next_day = datetime(int(y2), int(m2), int(d2))
    return abs((next_day - cur_day).days)

In [ ]:
n = 0
i = 1
ci = 1
date1 = neg_pos_hotel.iloc[1,3]
date2 = neg_pos_hotel.iloc[1,3]
for j in range(1,1093):
    if neg_pos_hotel.iloc[j,2][2:] == str(i):
        if neg_pos_hotel.iloc[j,4] == neg_pos_hotel.iloc[j-1,4]:
            date2 = neg_pos_hotel.iloc[j,3]
            ci = ci + 1
            if neg_pos_hotel.iloc[j,4] == '2018':
                q.iloc[n,0] = float(q.iloc[n,0]) + float(neg_pos_hotel.iloc[j,0])
                q.iloc[n,1] = float(q.iloc[n,1]) + float(neg_pos_hotel.iloc[j,1])
                q.iloc[n,2] = neg_pos_hotel.iloc[j,2]
                q.iloc[n,3] = '2018'
            elif neg_pos_hotel.iloc[j,4] == '2019':
                q.iloc[n,0] = float(q.iloc[n,0]) + float(neg_pos_hotel.iloc[j,0])
                q.iloc[n,1] = float(q.iloc[n,1]) + float(neg_pos_hotel.iloc[j,1])
                q.iloc[n,2] = neg_pos_hotel.iloc[j,2]
                q.iloc[n,3] = '2019'
            elif neg_pos_hotel.iloc[j,4] == '2020':
                q.iloc[n,0] = q.iloc[n,0] + float(neg_pos_hotel.iloc[j,0])
                q.iloc[n,1] = q.iloc[n,1] + float(neg_pos_hotel.iloc[j,1])
                q.iloc[n,2] = neg_pos_hotel.iloc[j,2]
                q.iloc[n,3] = '2020'
            elif neg_pos_hotel.iloc[j,4] == '2021':
                q.iloc[n,0] = q.iloc[n,0] + float(neg_pos_hotel.iloc[j,0])
                q.iloc[n,1] = q.iloc[n,1] + float(neg_pos_hotel.iloc[j,1])
                q.iloc[n,2] = data3.iloc[j,2]
                q.iloc[n,3] = '2021'
        else:
            q.iloc[n,5] = ci
            ci = 1
            result = daysBetweenDates(date1, date2)
            q.iloc[n,4] = result
            date1 = data3.iloc[j,3]
            date2 = data3.iloc[j,3]
            n = n + 1
            if data3.iloc[j,4] == '2018':
                q.iloc[n,0] = q.iloc[n,0] + float(neg_pos_hotel.iloc[j,0])
                q.iloc[n,1] = q.iloc[n,1] + float(neg_pos_hotel.iloc[j,1])
                q.iloc[n,2] = neg_pos_hotel.iloc[j,2]
                q.iloc[n,3] = '2018'
            elif data3.iloc[j,4] == '2019':
                q.iloc[n,0] = q.iloc[n,0] + float(neg_pos_hotel.iloc[j,0])
                q.iloc[n,1] = q.iloc[n,1] + float(neg_pos_hotel.iloc[j,1])
                q.iloc[n,2] = neg_pos_hotel.iloc[j,2]
                q.iloc[n,3] = '2019'
            elif neg_pos_hotel.iloc[j,4] == '2020':
                q.iloc[n,0] = q.iloc[n,0] + float(neg_pos_hotel.iloc[j,0])
                q.iloc[n,1] = q.iloc[n,1] + float(neg_pos_hotel.iloc[j,1])
                q.iloc[n,2] = neg_pos_hotel.iloc[j,2]
                q.iloc[n,3] = '2020'
            elif neg_pos_hotel.iloc[j,4] == '2021':
                q.iloc[n,0] = q.iloc[n,0] + float(neg_pos_hotel.iloc[j,0])
                q.iloc[n,1] = q.iloc[n,1] + float(neg_pos_hotel.iloc[j,1])
                q.iloc[n,2] = neg_pos_hotel.iloc[j,2]
                q.iloc[n,3] = '2021'
    else:
        q.iloc[n,5] = ci
        ci = 1
        result = daysBetweenDates(date1, date2)
        q.iloc[n,4] = result
        date1 = neg_pos_hotel.iloc[j,3]
        date2 = neg_pos_hotel.iloc[j,3]
        i = i + 1
        n = n + 1
        if neg_pos_hotel.iloc[j,4] == '2018':
            q.iloc[n,0] = q.iloc[n,0] + float(neg_pos_hotel.iloc[j,0])
            q.iloc[n,1] = q.iloc[n,1] + float(neg_pos_hotel.iloc[j,1])
            q.iloc[n,2] = data3.iloc[j,2]
            q.iloc[n,3] = '2018'
        elif data3.iloc[j,4] == '2019':
            q.iloc[n,0] = q.iloc[n,0] + float(data3.iloc[j,0])
            q.iloc[n,1] = q.iloc[n,1] + float(data3.iloc[j,1])
            q.iloc[n,2] = neg_pos_hotel.iloc[j,2]
            q.iloc[n,3] = '2019'
        elif neg_pos_hotel.iloc[j,4] == '2020':
            q.iloc[n,0] = q.iloc[n,0] + float(neg_pos_hotel.iloc[j,0])
            q.iloc[n,1] = q.iloc[n,1] + float(neg_pos_hotel.iloc[j,1])
            q.iloc[n,2] = neg_pos_hotel.iloc[j,2]
            q.iloc[n,3] = '2020'
        elif neg_pos_hotel.iloc[j,4] == '2021':
            q.iloc[n,0] = q.iloc[n,0] + float(neg_pos_hotel.iloc[j,0])
            q.iloc[n,1] = q.iloc[n,1] + float(neg_pos_hotel.iloc[j,1])
            q.iloc[n,2] = neg_pos_hotel.iloc[j,2]
            q.iloc[n,3] = '2021'
    

### 景区

In [26]:
view_pl_20 = pd.read_csv('数据\\2020_2021景区评论.csv',encoding='gbk').astype(str)#导入20年数据
view_pl_22 = pd.read_csv('数据\\2022_2023景区评论.csv',encoding='gbk').astype(str)#导入22年数据
view_pl= pd.concat([view_pl_20,view_pl_22])
view_pl

,景区评论ID,城市,景区名称,评论日期,评论内容
0,1001,茂名,中国第一滩旅游度假区,2018-01-01,超级美 冬天白日里最是舒服
1,1002,茂名,广垦（茂名）国家热带农业公园,2018-01-02,上了一节深刻的农业科普课，值了，在携程买价格挺合理，当时现场活动价35元。“氧吧”，“花海”...
2,1003,茂名,西江温泉度假村,2018-01-03,西江温泉度假村坐落于南国玉都——信宜市郊北界镇西江岸边，西江温泉故此得名，距市中心12公里，...
3,1004,茂名,石根山,2018-01-05,爬的很累！感觉还是挺不错的，设施再升级完善。2个多小时才爬到山顶！要是有玻璃栈道就完美了！
4,1005,茂名,天马山生态旅游区,2018-01-06,这次是和家人一起出游，选择天马山也是觉得比较适合老人和小朋友，游玩起来不会太累。其实早在04...
...,...,...,...,...,...
659,2199,茂名,茂名森林公园,2021-11-29,茂名森林公园不仅是旅游、观光、休闲、娱乐的好去处，更是展示生物多样性和奇妙性，传播百科知识，...
660,2200,茂名,湖光岩,2021-11-30,湖光岩风景区位于中国大陆最南端湛江市区西南18公里处，被联合国地质专家称为研究地球与地质科学...
661,2201,茂名,鼎龙湾欢乐海岸,2021-12-01,吴川市鼎龙湾附近一个免费的摄影基地，好看的场景 真的非常非常多，我因为没带几套衣服，时间也有...
662,2202,茂名,御水古温泉旅游度假区,2021-12-02,#广东旅游 #广东旅游攻略 #发现旅途的色彩 #冲啊浪人们 广东茂名旅游景点


In [27]:
#大致去重
reviews=view_pl.copy()
print('去重之前,',reviews.shape[0])
reviews_view=reviews.drop_duplicates([ '评论内容'])
print('去重之后,',reviews_view.shape[0])

去重之前, 1203
去重之后, 1193


In [28]:
reviews_view_id = reviews_view['景区评论ID']
reviews_view_name = reviews_view['景区名称']
reviews_view_pl = reviews_view['评论内容']
reviews_view_date =reviews_view['评论日期']
reviews_view = pd.concat((reviews_view_id,reviews_view_name,reviews_view_date,reviews_view_pl),axis=1,join ='inner')
reviews_view

,景区评论ID,景区名称,评论日期,评论内容
0,1001,中国第一滩旅游度假区,2018-01-01,超级美 冬天白日里最是舒服
1,1002,广垦（茂名）国家热带农业公园,2018-01-02,上了一节深刻的农业科普课，值了，在携程买价格挺合理，当时现场活动价35元。“氧吧”，“花海”...
2,1003,西江温泉度假村,2018-01-03,西江温泉度假村坐落于南国玉都——信宜市郊北界镇西江岸边，西江温泉故此得名，距市中心12公里，...
3,1004,石根山,2018-01-05,爬的很累！感觉还是挺不错的，设施再升级完善。2个多小时才爬到山顶！要是有玻璃栈道就完美了！
4,1005,天马山生态旅游区,2018-01-06,这次是和家人一起出游，选择天马山也是觉得比较适合老人和小朋友，游玩起来不会太累。其实早在04...
...,...,...,...,...
659,2199,茂名森林公园,2021-11-29,茂名森林公园不仅是旅游、观光、休闲、娱乐的好去处，更是展示生物多样性和奇妙性，传播百科知识，...
660,2200,湖光岩,2021-11-30,湖光岩风景区位于中国大陆最南端湛江市区西南18公里处，被联合国地质专家称为研究地球与地质科学...
661,2201,鼎龙湾欢乐海岸,2021-12-01,吴川市鼎龙湾附近一个免费的摄影基地，好看的场景 真的非常非常多，我因为没带几套衣服，时间也有...
662,2202,御水古温泉旅游度假区,2021-12-02,#广东旅游 #广东旅游攻略 #发现旅途的色彩 #冲啊浪人们 广东茂名旅游景点


In [29]:
reviews_view['景区评论ID'] = reviews_view['景区评论ID']+'-'+reviews_view['景区名称'].map(str)
reviews_view=reviews_view.sort_values(['景区名称'])#按照名称安排在一起
reviews_view_name_qc =reviews_view.drop_duplicates(['景区名称'])
reviews_view_name_qc

,景区评论ID,景区名称,评论日期,评论内容
65,1605-三官山,三官山,2020-03-22,三官山是值得去的地方。风景优美期待下次
281,1821-东海岛,东海岛,2020-11-04,东海岛景色优美，周围环境不错，酒店很多，也很实惠，吃海鲜很便宜的，值得一游的地方，停车方便的...
24,1564-中华第一冼太庙,中华第一冼太庙,2020-01-28,为什么我们姓冼的去看望祖先要收门票
58,1059-中国南极长城站,中国南极长城站,2018-03-11,长城站是我们在回程的最后一站 本来应该第一天就登陆的 但是由于天气原因一直拖到了最后 冒着遇...
181,1182-中国第一滩旅游度假区,中国第一滩旅游度假区,2018-09-18,2018年暑假全家去的，沙子很细，沙滩很平，很长，超爽！早晨，人少，沙滩漫步，孩子很喜欢！附...
...,...,...,...,...
316,1317-黎村温泉,黎村温泉,2019-04-20,温泉度假村环境不错，水质很好。
533,1534-鼎龙湾德萨斯水世界,鼎龙湾德萨斯水世界,2019-12-26,去过很多个水上世界，中规中矩吧，如果没有玩过的话，去玩玩也挺不错的，我去的时候还没完全建设好...
627,2167-鼎龙湾欢乐海岸,鼎龙湾欢乐海岸,2021-10-27,#湛江怎么玩 广东周边游湛江鼎龙湾周末好去处海边骑马|||发现了一个小众海滩，人少海滩干净，...
398,1399-鼎龙湾高尔夫度假村,鼎龙湾高尔夫度假村,2019-07-26,湛江鼎龙湾度假村是新开发的，所以现人还不多，环境得漂亮，有得长很漂亮的沙滩，水上乐乐园！


In [30]:
print('去重之前景区数量,',reviews_view.shape[0])
print('去重之后景区数量,',reviews_view_name_qc.shape[0])

去重之前景区数量, 1193
去重之后景区数量, 113


In [31]:
view_name = reviews_view_name_qc['景区名称']
view_name.to_csv('景区.csv',encoding='utf_8_sig',index=False)

In [32]:
view_number = list(range(90,203))
connect_view = dict(zip(view_name,view_number))#组成字典
reviews_view['产品ID']= 0
for i in range(1193):
    I = reviews_view.iloc[i,1]
    reviews_view.iloc[i,4] = connect_view[I]

In [33]:
reviews_view['ID'] = 'ID'
reviews_view['产品ID'] = reviews_view['ID'].map(str) + reviews_view['产品ID'].map(str)
reviews_view

,景区评论ID,景区名称,评论日期,评论内容,产品ID,ID
65,1605-三官山,三官山,2020-03-22,三官山是值得去的地方。风景优美期待下次,ID90,ID
281,1821-东海岛,东海岛,2020-11-04,东海岛景色优美，周围环境不错，酒店很多，也很实惠，吃海鲜很便宜的，值得一游的地方，停车方便的...,ID91,ID
205,1206-东海岛,东海岛,2018-11-04,嗯嗯，东海岛的风景迷人，真的很好玩。我超喜欢，值得去玩玩，看看，走走。.,ID91,ID
63,1603-东海岛,东海岛,2020-03-20,风，那么轻柔，带动着小树、小草一起翩翩起舞，当一阵清风飘来，如同母亲的手轻轻抚摸自己的脸庞，...,ID91,ID
62,1602-东海岛,东海岛,2020-03-19,有空就过来这里逛逛，这里很美很舒服的！,ID91,ID
...,...,...,...,...,...,...
200,1740-鼎龙湾欢乐海岸,鼎龙湾欢乐海岸,2020-08-13,娱乐项目挺多的，很不错的地方，而且离家很近，开车很快就到了，很不错哦！,ID200,ID
398,1399-鼎龙湾高尔夫度假村,鼎龙湾高尔夫度假村,2019-07-26,湛江鼎龙湾度假村是新开发的，所以现人还不多，环境得漂亮，有得长很漂亮的沙滩，水上乐乐园！,ID201,ID
35,1036-龙海天,龙海天,2018-02-12,湛江市区最近最长的海滩，海沙细幼清洁。,ID202,ID
68,1069-龙海天,龙海天,2018-03-23,龙海天距离市区很远，但是从市区到龙海天很多直达车，霞山，赤坎都有到龙海天的直达车，途中可以穿...,ID202,ID


#### 评论内容清洗

In [34]:
view = reviews_view['评论内容'].progress_apply(stopWord)#去停用词

100%|███████████████████████████████████████████████████████████████████████████| 1193/1193 [00:00<00:00, 21279.11it/s]


In [35]:
view = reviews_view['评论内容'].progress_apply(cleanmytxt)#process_apply为进度条

100%|████████████████████████████████████████████████████████████████████████████| 1193/1193 [00:00<00:00, 2204.63it/s]


In [36]:
reviews_view['分词并去停用词'] = view

In [37]:
reviews_view

,景区评论ID,景区名称,评论日期,评论内容,产品ID,ID,分词并去停用词
65,1605-三官山,三官山,2020-03-22,三官山是值得去的地方。风景优美期待下次,ID90,ID,三 官山 是 值得 去 的 地方 。 风景优美 期待 下次
281,1821-东海岛,东海岛,2020-11-04,东海岛景色优美，周围环境不错，酒店很多，也很实惠，吃海鲜很便宜的，值得一游的地方，停车方便的...,ID91,ID,东海岛 景色 优美 ， 周围环境 不错 ， 酒店 很多 ， 也 很 实惠 ， 吃海鲜 很 便...
205,1206-东海岛,东海岛,2018-11-04,嗯嗯，东海岛的风景迷人，真的很好玩。我超喜欢，值得去玩玩，看看，走走。.,ID91,ID,嗯 嗯 ， 东海岛 的 风景迷人 ， 真的 很 好玩 。 我超 喜欢 ， 值得 去 玩玩 ，...
63,1603-东海岛,东海岛,2020-03-20,风，那么轻柔，带动着小树、小草一起翩翩起舞，当一阵清风飘来，如同母亲的手轻轻抚摸自己的脸庞，...,ID91,ID,风 ， 那么 轻柔 ， 带动 着 小树 、 小草 一起 翩翩起舞 ， 当 一阵 清风 飘 来...
62,1602-东海岛,东海岛,2020-03-19,有空就过来这里逛逛，这里很美很舒服的！,ID91,ID,有空 就 过来 这里 逛逛 ， 这里 很美 很 舒服 的 ！
...,...,...,...,...,...,...,...
200,1740-鼎龙湾欢乐海岸,鼎龙湾欢乐海岸,2020-08-13,娱乐项目挺多的，很不错的地方，而且离家很近，开车很快就到了，很不错哦！,ID200,ID,娱乐 项目 挺 多 的 ， 很 不错 的 地方 ， 而且 离家 很近 ， 开车 很快 就 到...
398,1399-鼎龙湾高尔夫度假村,鼎龙湾高尔夫度假村,2019-07-26,湛江鼎龙湾度假村是新开发的，所以现人还不多，环境得漂亮，有得长很漂亮的沙滩，水上乐乐园！,ID201,ID,湛江 鼎 龙湾 度假村 是 新 开发 的 ， 所以 现人 还 不 多 ， 环境 得 漂亮 ，...
35,1036-龙海天,龙海天,2018-02-12,湛江市区最近最长的海滩，海沙细幼清洁。,ID202,ID,湛江市 区 最近 最长 的 海滩 ， 海沙 细幼 清洁 。
68,1069-龙海天,龙海天,2018-03-23,龙海天距离市区很远，但是从市区到龙海天很多直达车，霞山，赤坎都有到龙海天的直达车，途中可以穿...,ID202,ID,龙 海天 距离 市区 很 远 ， 但是 从 市区 到 龙 海天 很多 直达车 ， 霞山 ， ...


### 餐饮

In [38]:
restaurant_pl_20 = pd.read_csv('数据\\2020_2021餐饮评论.csv',encoding='gbk').astype(str)#导入20年数据
restaurant_pl_22 = pd.read_csv('数据\\2022_2023餐饮评论.csv',encoding='gbk').astype(str)#导入22年数据
restaurant_pl = pd.concat([restaurant_pl_20,restaurant_pl_22])
restaurant_pl

,餐饮评论ID,城市,餐饮名称,评论日期,评论内容,标题
0,1001,茂名,盛香烧鹅（东方市场店）,2018-01-01,很好吃推荐！,主食3选1，有赠品
1,1002,茂名,清香面包店（车田街店）,2018-01-01,超级好吃 老板又好 量又多,水果忌廉夹心蛋糕（二层）1个，约4磅，圆
2,1003,茂名,功夫鸡排（光华北路店）,2018-01-01,好吃，不得不说比门口正对面那家好吃。服务态度也好。炸的皮松软，不会炸得很硬。,小吃6选1，提供免费WiFi
3,1004,茂名,茂名浪漫海岸温德姆酒店望海餐厅,2018-01-02,品种少，冷冻食品多。 7点就不再出食物，难以接受。,自助晚餐（浪漫海岸跨年音乐节）
4,1005,茂名,清香面包店（车田街店）,2018-01-03,味道不错，至少吃起来不腻，并且全家7个人吃还是挺多的了，就比较实惠咯,水果忌廉夹心蛋糕（二层）1个，约4磅，圆
...,...,...,...,...,...,...
5802,7980,茂名,围炉海鲜烤肉自助（金港湾店）,2021-12-19,吃得好饱，用餐感觉很好，下次还会继续来,晚餐/节假日全天
5803,7981,茂名,围炉海鲜烤肉自助（金港湾店）,2021-12-20,回头客了，高州只喜欢来这里,工作日午餐
5804,7982,茂名,围炉海鲜烤肉自助（金港湾店）,2021-12-20,很好吃,工作日午餐
5805,7983,茂名,围炉海鲜烤肉自助（金港湾店）,2021-12-21,非常好,【学生】晚餐/节假日全天


In [39]:
restaurant_pl.info()#查看属性

<class 'pandas.core.frame.DataFrame'>
Index: 6984 entries, 0 to 5806
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   餐饮评论ID  6984 non-null   object
 1   城市      6984 non-null   object
 2   餐饮名称    6984 non-null   object
 3   评论日期    6984 non-null   object
 4   评论内容    6984 non-null   object
 5   标题      6984 non-null   object
dtypes: object(6)
memory usage: 381.9+ KB


In [40]:
#打印每个值的出现次数
for counts in ['餐饮评论ID','餐饮名称','评论日期','评论内容','标题']:
    restaurant_pl[counts].value_counts()
print(restaurant_pl[counts].value_counts())

标题
100元代金券1张，可叠加2张     348
可叠加，除饮料，酒水外全场通用     197
100元代金券1张，可叠加       195
贵族自助牛排套餐，建议单人使用     181
精品浪漫双人餐1份           175
                   ... 
咖啡下午单人餐，包间免费          1
海盐鲜果杯1个，约1杯，杯         1
红豆双皮奶1杯，提供免费WiFi      1
支竹牛腩饭+香滑蒸蛋1份          1
公主款摆件蛋糕1个，约6英寸，圆      1
Name: count, Length: 558, dtype: int64


In [41]:
#大致去重
reviews=restaurant_pl.copy()
print('去重之前,',reviews.shape[0])
reviews=reviews.drop_duplicates([ '评论内容'])
print('去重之后,',reviews.shape[0])

去重之前, 6984
去重之后, 6227


In [42]:
reviews_id = reviews['餐饮评论ID']
reviews_name = reviews['餐饮名称']
reviews_pl = reviews['评论内容']
reviews_date =reviews['评论日期']
reviews_res = pd.concat((reviews_id,reviews_name,reviews_date,reviews_pl),axis=1,join ='inner')
reviews_res

,餐饮评论ID,餐饮名称,评论日期,评论内容
0,1001,盛香烧鹅（东方市场店）,2018-01-01,很好吃推荐！
1,1002,清香面包店（车田街店）,2018-01-01,超级好吃 老板又好 量又多
2,1003,功夫鸡排（光华北路店）,2018-01-01,好吃，不得不说比门口正对面那家好吃。服务态度也好。炸的皮松软，不会炸得很硬。
3,1004,茂名浪漫海岸温德姆酒店望海餐厅,2018-01-02,品种少，冷冻食品多。 7点就不再出食物，难以接受。
4,1005,清香面包店（车田街店）,2018-01-03,味道不错，至少吃起来不腻，并且全家7个人吃还是挺多的了，就比较实惠咯
...,...,...,...,...
5799,7977,围炉海鲜烤肉自助（金港湾店）,2021-12-18,我们5个帅哥都说好吃
5801,7979,围炉海鲜烤肉自助（金港湾店）,2021-12-18,还可以哦
5802,7980,围炉海鲜烤肉自助（金港湾店）,2021-12-19,吃得好饱，用餐感觉很好，下次还会继续来
5803,7981,围炉海鲜烤肉自助（金港湾店）,2021-12-20,回头客了，高州只喜欢来这里


In [43]:
reviews_res['餐饮评论ID'] = reviews_res['餐饮评论ID']+'-'+reviews_res['餐饮名称'].map(str)
reviews_res=reviews_res.sort_values(['餐饮名称'])#按照名称安排在一起
reviews_res_name_qc =reviews_res.drop_duplicates(['餐饮名称'])
reviews_res_name_qc

,餐饮评论ID,餐饮名称,评论日期,评论内容
3655,5833-CAKE情迷黑森林,CAKE情迷黑森林,2021-07-16,水果布丁蛋糕很好吃 服务态度也很不错 位置也很容易找 一下子就找到了 料有很多 下次再来这边订蛋糕
796,2974-Hello炸鸡（方兴店）,Hello炸鸡（方兴店）,2020-09-26,环境很好，老板特别好，炸鸡味道也不错。
1694,3872-LINLEE·手打柠檬茶（文创店）,LINLEE·手打柠檬茶（文创店）,2021-02-22,好喝值得一试环境也不错同学连续三天喝柠檬茶
4618,6796-三两粉（茂名东汇城店）,三两粉（茂名东汇城店）,2021-09-08,青椒量有点夸张啊！不能吃辣的可别点这个！
77,1078-丰年面包店（茂南区店）,丰年面包店（茂南区店）,2018-02-04,栗子蛋糕挺好吃的，因为太激动所以没有拍照片一下子吃光了，然后服务态度也挺好，接电话的人声音挺好听的
...,...,...,...,...
984,1985-鲜滋味蛋糕·季念日生日蛋糕（茂南区店）,鲜滋味蛋糕·季念日生日蛋糕（茂南区店）,2019-08-07,榴莲雪媚娘很好吃，很香味道不错棒棒哒
1947,4125-鲜饮时光（高州店）,鲜饮时光（高州店）,2021-03-27,不好吃 不好吃 紫菜特别难咬断 没有紫菜味道 饭很多水没有颗粒感
4022,6200-麦壳西点（开发区店）,麦壳西点（开发区店）,2021-08-06,蛋糕很新鲜也非常好吃，奶油很香，包装很好看店长小哥也好好呀，态度很好，也很帅哦，超级大方（送...
2192,4370-麦当劳（明湖店）,麦当劳（明湖店）,2021-04-16,新货还不错


In [44]:
print('去重之前餐饮店铺数量,',reviews_res.shape[0])
print('去重之后餐饮店铺数量,',reviews_res_name_qc.shape[0])

去重之前餐饮店铺数量, 6227
去重之后餐饮店铺数量, 79


In [45]:
restaurant_name = reviews_res_name_qc['餐饮名称']
restaurant_name.to_csv('餐饮.csv',encoding='utf_8_sig',index=False)

In [46]:
restaurant_number = list(range(203,282))
connect_restaurant = dict(zip(restaurant_name,restaurant_number))
reviews_res['产品ID']= 0
for i in range(6227):
    I = reviews_res.iloc[i,1]
    reviews_res.iloc[i,4] = connect_restaurant[I]

In [47]:
reviews_res['ID'] = 'ID'
reviews_res['产品ID'] = reviews_res['ID'].map(str) + reviews_res['产品ID'].map(str)
reviews_res

,餐饮评论ID,餐饮名称,评论日期,评论内容,产品ID,ID
3655,5833-CAKE情迷黑森林,CAKE情迷黑森林,2021-07-16,水果布丁蛋糕很好吃 服务态度也很不错 位置也很容易找 一下子就找到了 料有很多 下次再来这边订蛋糕,ID203,ID
230,2408-CAKE情迷黑森林,CAKE情迷黑森林,2020-06-01,味道还不错！就是跟图片上的友差异……,ID203,ID
772,1773-CAKE情迷黑森林,CAKE情迷黑森林,2019-02-20,每次去买到的泡芙都是脆脆的很好吃，这次的竟然都是软的，不好吃，还影响口感。失望,ID203,ID
773,1774-CAKE情迷黑森林,CAKE情迷黑森林,2019-02-20,榴莲千层每次都是吃这家！,ID203,ID
776,1777-CAKE情迷黑森林,CAKE情迷黑森林,2019-02-21,，，，， ，，，，，，，，，，，，，，，，,ID203,ID
...,...,...,...,...,...,...
3289,5467-黑色经典长沙臭豆腐（为民路店）,黑色经典长沙臭豆腐（为民路店）,2021-06-26,臭豆腐的味道很不错，选的椰果奶茶就一般般了。,ID281,ID
3741,5919-黑色经典长沙臭豆腐（为民路店）,黑色经典长沙臭豆腐（为民路店）,2021-07-21,冻柠七刚运动完 喝起来很爽带点咸味臭豆腐味道不错 还能做得外脆里嫩 店家继续加油,ID281,ID
3713,5891-黑色经典长沙臭豆腐（为民路店）,黑色经典长沙臭豆腐（为民路店）,2021-07-19,很好吃 肠胃好的朋友可以多吃些,ID281,ID
4007,6185-黑色经典长沙臭豆腐（为民路店）,黑色经典长沙臭豆腐（为民路店）,2021-08-05,感觉是茂南市区的top了哈哈哈,ID281,ID


#### 评论内容清洗

In [48]:
restaurant = reviews_res['评论内容'].progress_apply(cleanmytxt)#process_apply为进度条

100%|████████████████████████████████████████████████████████████████████████████| 6227/6227 [00:01<00:00, 3669.44it/s]


In [49]:
restaurant = restaurant.progress_apply(stopWord)#去停用词

100%|████████████████████████████████████████████████████████████████████████████| 6227/6227 [00:02<00:00, 2819.83it/s]


In [50]:
reviews_res['分词并去停用词'] = restaurant

In [51]:
reviews_res

,餐饮评论ID,餐饮名称,评论日期,评论内容,产品ID,ID,分词并去停用词
3655,5833-CAKE情迷黑森林,CAKE情迷黑森林,2021-07-16,水果布丁蛋糕很好吃 服务态度也很不错 位置也很容易找 一下子就找到了 料有很多 下次再来这边订蛋糕,ID203,ID,水果 布丁 蛋糕 好吃 服务态度 不错 位置 找 找到 料 很多 下次 订 蛋糕
230,2408-CAKE情迷黑森林,CAKE情迷黑森林,2020-06-01,味道还不错！就是跟图片上的友差异……,ID203,ID,味道 不错 图片 友 差异 … …
772,1773-CAKE情迷黑森林,CAKE情迷黑森林,2019-02-20,每次去买到的泡芙都是脆脆的很好吃，这次的竟然都是软的，不好吃，还影响口感。失望,ID203,ID,每次 买 泡 芙 脆脆 好吃 这次 竟然 软 好吃 影响 口感 失望
773,1774-CAKE情迷黑森林,CAKE情迷黑森林,2019-02-20,榴莲千层每次都是吃这家！,ID203,ID,榴莲 千层 每次 吃 这家
776,1777-CAKE情迷黑森林,CAKE情迷黑森林,2019-02-21,，，，， ，，，，，，，，，，，，，，，，,ID203,ID,
...,...,...,...,...,...,...,...
3289,5467-黑色经典长沙臭豆腐（为民路店）,黑色经典长沙臭豆腐（为民路店）,2021-06-26,臭豆腐的味道很不错，选的椰果奶茶就一般般了。,ID281,ID,臭豆腐 味道 不错 选 椰果 奶茶 一般般
3741,5919-黑色经典长沙臭豆腐（为民路店）,黑色经典长沙臭豆腐（为民路店）,2021-07-21,冻柠七刚运动完 喝起来很爽带点咸味臭豆腐味道不错 还能做得外脆里嫩 店家继续加油,ID281,ID,冻 柠七刚 运动 完喝 很爽 带点 咸味 臭豆腐 味道 不错 外脆 嫩 店家 加油
3713,5891-黑色经典长沙臭豆腐（为民路店）,黑色经典长沙臭豆腐（为民路店）,2021-07-19,很好吃 肠胃好的朋友可以多吃些,ID281,ID,好吃 肠胃 好 朋友 多吃些
4007,6185-黑色经典长沙臭豆腐（为民路店）,黑色经典长沙臭豆腐（为民路店）,2021-08-05,感觉是茂南市区的top了哈哈哈,ID281,ID,感觉 茂 南市区 top 哈哈哈


# 2.LDA主题词典构建

## 酒店tfidf-lda

In [52]:
#变量
# 文本 csv 文件里面文本所处的列名,注意这里一定要填对，要不然会报错的！！！
document_column_name_hotel = '回答内容'
# 输出主题词的文件路径
top_words_csv_path_hotel = 'top-topic-words-hotel.csv'
# 输出各文档所属主题的文件路径
predict_topic_csv_path_hotel= 'document-distribution-hotel.csv'
# 可视化 html 文件路径
html_path_hotel = 'document-lda-visualization-hotel.html'
# 选定的主题数
n_topics_hotel = 3
# 要输出的每个主题的前 n_top_words 个主题词数
n_top_words_hotel = 20
# 去除无意义字符的正则表达式
pattern_hotel = u'[\\s\\d,.<>/?:;\'\"[\\]{}()\\|~!\t"@#$%^&*\\-_=+，。\n《》、？：；“”‘’｛｝【】（）…￥！—┄－]+'

In [53]:
# 构造 TF-IDF
tf_idf_vectorizer_hotel = TfidfVectorizer()
tf_idf_hotel = tf_idf_vectorizer_hotel.fit_transform(reviews_hotel['分词并去停用词'])

In [54]:
# 特征词列表
feature_names_hotel = tf_idf_vectorizer_hotel.get_feature_names_out() 
# 特征词 TF-IDF 矩阵
matrix_hotel = tf_idf_hotel.toarray()
feature_names_df_hotel = pd.DataFrame(matrix_hotel,columns=feature_names_hotel)
print(feature_names_df_hotel)
feature_names_df_hotel

       00  002  008        02  028   10  100  10ms   11  110  ...   鸭子   麻烦  \
0     0.0  0.0  0.0  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0   
1     0.0  0.0  0.0  0.611286  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0   
2     0.0  0.0  0.0  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0   
3     0.0  0.0  0.0  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0   
4     0.0  0.0  0.0  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0   
...   ...  ...  ...       ...  ...  ...  ...   ...  ...  ...  ...  ...  ...   
1081  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0   
1082  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0   
1083  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0   
1084  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0   
1085  0.0  0.0  0.0  0.000000  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0   

       黄金   默默   鼓励  鼠标垫   齐全   齐声   齐备   龙眼  
0   

,00,002,008,02,028,10,100,10ms,11,110,...,鸭子,麻烦,黄金,默默,鼓励,鼠标垫,齐全,齐声,齐备,龙眼
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.611286,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1082,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1083,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1084,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
##LDA模型
from sklearn.decomposition import LatentDirichletAllocation
# 指定 lda 主题数
n_topics = 3
lda = LatentDirichletAllocation(
    n_components=n_topics, max_iter=50,
    learning_method='online',
    learning_offset=50.,
    random_state=0)
print(lda)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, n_components=3, random_state=0)


In [56]:
# 给 LDA 喂生成的 TF-IDF 矩阵
lda.fit(tf_idf_hotel)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, n_components=3, random_state=0)

In [57]:
#将lda结果转化为df
def top_words_data_frame(model: LatentDirichletAllocation,
                         tf_idf_vectorizer: TfidfVectorizer,
                         n_top_words: int) -> pd.DataFrame:
    '''
    求出每个主题的前 n_top_words 个词

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    tf_idf_vectorizer : sklearn 的 TfidfVectorizer
    n_top_words :前 n_top_words 个主题词

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    rows = []
    feature_names = tf_idf_vectorizer.get_feature_names_out() 
    for topic in model.components_:
        top_words = [feature_names[i]
                     for i in topic.argsort()[:-n_top_words - 1:-1]]
        rows.append(top_words)
    columns = [f'topic {i+1}' for i in range(n_top_words)]
    df = pd.DataFrame(rows, columns=columns)

    return df


def predict_to_data_frame(model: LatentDirichletAllocation, X: np.ndarray) -> pd.DataFrame:
    '''
    求出文档主题概率分布情况

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    X : 词向量矩阵

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    # 求出给定文档的主题概率分布矩阵
    matrix = model.transform(X)
    columns = [f'P(topic {i+1})' for i in range(len(model.components_))]
    df = pd.DataFrame(matrix, columns=columns)
    return df

In [58]:
# 计算 n_top_words 个主题词
top_words_df_hotel = top_words_data_frame(lda, tf_idf_vectorizer_hotel, n_top_words_hotel)

# 保存 n_top_words 个主题词到 csv 文件中
top_words_df_hotel.to_csv(top_words_csv_path_hotel, encoding='utf-8-sig', index=None)

# 转 tf_idf 为数组，以便后面使用它来对文本主题概率分布进行计算
X = tf_idf_hotel.toarray()

# 计算完毕主题概率分布情况
predict_df_hotel = predict_to_data_frame(lda, X)

# 保存文本主题概率分布到 csv 文件中
predict_df_hotel.to_csv(predict_topic_csv_path_hotel, encoding='utf-8-sig', index=None)

# 使用 pyLDAvis 进行可视化
data = pyLDAvis.lda_model.prepare(lda, tf_idf_hotel, tf_idf_vectorizer_hotel)
pyLDAvis.save_html(data, html_path_hotel)
# 清屏
os.system('clear')
# 浏览器打开 html 文件以查看可视化结果
os.system(f'start {html_path_hotel}')

print('本次生成了文件：',
      top_words_csv_path_hotel,
      predict_topic_csv_path_hotel,
      html_path_hotel)

本次生成了文件： top-topic-words-hotel.csv document-distribution-hotel.csv document-lda-visualization-hotel.html


## 景区tfidf-lda

In [59]:
#变量
# 文本 csv 文件里面文本所处的列名,注意这里一定要填对，要不然会报错的！！！
document_column_name_view = '回答内容'
# 输出主题词的文件路径
top_words_csv_path_view = 'top-topic-words-view.csv'
# 输出各文档所属主题的文件路径
predict_topic_csv_path_view= 'document-distribution-view.csv'
# 可视化 html 文件路径
html_path_view = 'document-lda-visualization-view.html'
# 选定的主题数
n_topics_view = 3
# 要输出的每个主题的前 n_top_words 个主题词数
n_top_words_view = 20
# 去除无意义字符的正则表达式
pattern_view = u'[\\s\\d,.<>/?:;\'\"[\\]{}()\\|~!\t"@#$%^&*\\-_=+，。\n《》、？：；“”‘’｛｝【】（）…￥！—┄－]+'

In [60]:
# 构造 TF-IDF
tf_idf_vectorizer_view = TfidfVectorizer()
tf_idf_view = tf_idf_vectorizer_view.fit_transform(reviews_view['分词并去停用词'])

In [61]:
# 特征词列表
feature_names_view = tf_idf_vectorizer_view.get_feature_names_out() 
# 特征词 TF-IDF 矩阵
matrix_view = tf_idf_view.toarray()
feature_names_df_view = pd.DataFrame(matrix_view,columns=feature_names_view)
print(feature_names_df_view)
feature_names_df_view

       00  00tips   04   07   09   10  100  1000  1007  101  ...   鼎悦  鼎湖山  \
0     0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...  0.0  0.0   
1     0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...  0.0  0.0   
2     0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...  0.0  0.0   
3     0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...  0.0  0.0   
4     0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...  0.0  0.0   
...   ...     ...  ...  ...  ...  ...  ...   ...   ...  ...  ...  ...  ...   
1188  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...  0.0  0.0   
1189  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...  0.0  0.0   
1190  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...  0.0  0.0   
1191  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...  0.0  0.0   
1192  0.0     0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...  0.0  0.0   

       鼎龙   鼓足   鼓韵   齐全   龙光        龙湾   龙舞   龙鱼  
0     0.0  

,00,00tips,04,07,09,10,100,1000,1007,101,...,鼎悦,鼎湖山,鼎龙,鼓足,鼓韵,齐全,龙光,龙湾,龙舞,龙鱼
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.288333,0.0,0.0
1190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [62]:
##LDA模型
from sklearn.decomposition import LatentDirichletAllocation
# 指定 lda 主题数
n_topics = 3
lda = LatentDirichletAllocation(
    n_components=n_topics, max_iter=50,
    learning_method='online',
    learning_offset=50.,
    random_state=0)
print(lda)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, n_components=3, random_state=0)


In [63]:
# 核心，给 LDA 喂生成的 TF-IDF 矩阵
lda.fit(tf_idf_view)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, n_components=3, random_state=0)

In [64]:
#将lda结果转化为df
def top_words_data_frame(model: LatentDirichletAllocation,
                         tf_idf_vectorizer: TfidfVectorizer,
                         n_top_words: int) -> pd.DataFrame:
    '''
    求出每个主题的前 n_top_words 个词

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    tf_idf_vectorizer : sklearn 的 TfidfVectorizer
    n_top_words :前 n_top_words 个主题词

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    rows = []
    feature_names = tf_idf_vectorizer.get_feature_names_out() 
    for topic in model.components_:
        top_words = [feature_names[i]
                     for i in topic.argsort()[:-n_top_words - 1:-1]]
        rows.append(top_words)
    columns = [f'topic {i+1}' for i in range(n_top_words)]
    df = pd.DataFrame(rows, columns=columns)

    return df


def predict_to_data_frame(model: LatentDirichletAllocation, X: np.ndarray) -> pd.DataFrame:
    '''
    求出文档主题概率分布情况

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    X : 词向量矩阵

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    # 求出给定文档的主题概率分布矩阵
    matrix = model.transform(X)
    columns = [f'P(topic {i+1})' for i in range(len(model.components_))]
    df = pd.DataFrame(matrix, columns=columns)
    return df

In [65]:
# 计算 n_top_words 个主题词
top_words_df_view = top_words_data_frame(lda, tf_idf_vectorizer_view, n_top_words_view)

# 保存 n_top_words 个主题词到 csv 文件中
top_words_df_view.to_csv(top_words_csv_path_view, encoding='utf-8-sig', index=None)

# 转 tf_idf 为数组，以便后面使用它来对文本主题概率分布进行计算
X = tf_idf_view.toarray()

# 计算完毕主题概率分布情况
predict_df_view = predict_to_data_frame(lda, X)

# 保存文本主题概率分布到 csv 文件中
predict_df_view.to_csv(predict_topic_csv_path_view, encoding='utf-8-sig', index=None)

# 使用 pyLDAvis 进行可视化
data = pyLDAvis.lda_model.prepare(lda, tf_idf_view, tf_idf_vectorizer_view)
pyLDAvis.save_html(data, html_path_view)
# 清屏
os.system('clear')
# 浏览器打开 html 文件以查看可视化结果
os.system(f'start {html_path_view}')

print('本次生成了文件：',
      top_words_csv_path_view,
      predict_topic_csv_path_view,
      html_path_view)

本次生成了文件： top-topic-words-view.csv document-distribution-view.csv document-lda-visualization-view.html


## 餐饮tfidf-lda

In [66]:
#变量
# 文本 csv 文件里面文本所处的列名,注意这里一定要填对，要不然会报错的！！！
document_column_name_restaurant = '回答内容'
# 输出主题词的文件路径
top_words_csv_path_restaurant = 'top-topic-words-restaurant.csv'
# 输出各文档所属主题的文件路径
predict_topic_csv_path_restaurant= 'document-distribution-restaurant.csv'
# 可视化 html 文件路径
html_path_restaurant = 'document-lda-visualization-restaurant.html'
# 选定的主题数
n_topics_restaurant = 3
# 要输出的每个主题的前 n_top_words 个主题词数
n_top_words_restaurant = 20
# 去除无意义字符的正则表达式
pattern_restaurant = u'[\\s\\d,.<>/?:;\'\"[\\]{}()\\|~!\t"@#$%^&*\\-_=+，。\n《》、？：；“”‘’｛｝【】（）…￥！—┄－]+'

In [67]:
# 构造 TF-IDF
tf_idf_vectorizer_restaurant = TfidfVectorizer()
tf_idf_restaurant = tf_idf_vectorizer_restaurant.fit_transform(reviews_res['分词并去停用词'])

In [68]:
# 特征词列表
feature_names_restaurant = tf_idf_vectorizer_restaurant.get_feature_names_out() 
# 特征词 TF-IDF 矩阵
matrix_restaurant = tf_idf_restaurant.toarray()
feature_names_df_restaurant = pd.DataFrame(matrix_restaurant,columns=feature_names_restaurant)
print(feature_names_df_restaurant)
feature_names_df_restaurant

       00  0416  088   09   10  100  100000000000000000  108   11  110  ...  \
0     0.0   0.0  0.0  0.0  0.0  0.0                 0.0  0.0  0.0  0.0  ...   
1     0.0   0.0  0.0  0.0  0.0  0.0                 0.0  0.0  0.0  0.0  ...   
2     0.0   0.0  0.0  0.0  0.0  0.0                 0.0  0.0  0.0  0.0  ...   
3     0.0   0.0  0.0  0.0  0.0  0.0                 0.0  0.0  0.0  0.0  ...   
4     0.0   0.0  0.0  0.0  0.0  0.0                 0.0  0.0  0.0  0.0  ...   
...   ...   ...  ...  ...  ...  ...                 ...  ...  ...  ...  ...   
6222  0.0   0.0  0.0  0.0  0.0  0.0                 0.0  0.0  0.0  0.0  ...   
6223  0.0   0.0  0.0  0.0  0.0  0.0                 0.0  0.0  0.0  0.0  ...   
6224  0.0   0.0  0.0  0.0  0.0  0.0                 0.0  0.0  0.0  0.0  ...   
6225  0.0   0.0  0.0  0.0  0.0  0.0                 0.0  0.0  0.0  0.0  ...   
6226  0.0   0.0  0.0  0.0  0.0  0.0                 0.0  0.0  0.0  0.0  ...   

      默默地  鼓鼓的   鼻水   鼾鼻   齐全   齐菜   齐飞  龙卷风   龙虾  

,00,0416,088,09,10,100,100000000000000000,108,11,110,...,默默地,鼓鼓的,鼻水,鼾鼻,齐全,齐菜,齐飞,龙卷风,龙虾,龟苓膏
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
##LDA模型
from sklearn.decomposition import LatentDirichletAllocation
# 指定 lda 主题数
n_topics = 3
lda = LatentDirichletAllocation(
    n_components=n_topics, max_iter=50,
    learning_method='online',
    learning_offset=50.,
    random_state=0)
print(lda)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, n_components=3, random_state=0)


In [70]:
# 核心，给 LDA 喂生成的 TF-IDF 矩阵
lda.fit(tf_idf_restaurant)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=50, n_components=3, random_state=0)

In [71]:
# 计算 n_top_words 个主题词
top_words_df_restaurant = top_words_data_frame(lda, tf_idf_vectorizer_restaurant, n_top_words_restaurant)

# 保存 n_top_words 个主题词到 csv 文件中
top_words_df_restaurant.to_csv(top_words_csv_path_restaurant, encoding='utf-8-sig', index=None)

# 转 tf_idf 为数组，以便后面使用它来对文本主题概率分布进行计算
X = tf_idf_restaurant.toarray()

# 计算完毕主题概率分布情况
predict_df_restaurant = predict_to_data_frame(lda, X)

# 保存文本主题概率分布到 csv 文件中
predict_df_restaurant.to_csv(predict_topic_csv_path_restaurant, encoding='utf-8-sig', index=None)

# 使用 pyLDAvis 进行可视化
data = pyLDAvis.lda_model.prepare(lda, tf_idf_restaurant, tf_idf_vectorizer_restaurant)
pyLDAvis.save_html(data, html_path_restaurant)
# 清屏
os.system('clear')
# 浏览器打开 html 文件以查看可视化结果
os.system(f'start {html_path_restaurant}')

print('本次生成了文件：',
      top_words_csv_path_restaurant,
      predict_topic_csv_path_restaurant,
      html_path_restaurant)

本次生成了文件： top-topic-words-restaurant.csv document-distribution-restaurant.csv document-lda-visualization-restaurant.html


# 游记评论的关联分析